In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
import os
cwd = os.getcwd()
print(cwd)

/home/ubuntu/work/saved_data/Global-Canopy-Height-Map


In [3]:
#samples = pd.read_csv(os.path.join(cwd, 'samples.csv'))
samples = pd.read_csv('/home/ubuntu/work/satellite_data/sentinel_pauls_paper/samples.csv')
print(samples.head())
#cwd = '/home/ubuntu/work/satellite_data/sentinel_pauls_paper'

   Unnamed: 0  g5  g10  g15  g20  g25  g30  totals    longitudes  \
0           0   0    0    0    0    0    0      64  7.406728e+06   
1           1   3    0    0    0    0    0       3  4.268613e+06   
2           2   0    0    0    0    0    0       1  4.275268e+06   
3           3  15    0    0    0    0    0      16  3.915071e+06   
4           4   8    0    0    0    0    0      56  3.945878e+06   

       latitudes                                         transforms  utm  \
0  296618.124489  | 10.00, 0.00, 293870.00|\n| 0.00,-10.00, 7406...  39S   
1  302341.260358  | 10.00, 0.00, 297320.00|\n| 0.00,-10.00, 4268...  37N   
2  278279.291022  | 10.00, 0.00, 277210.00|\n| 0.00,-10.00, 4278...  37N   
3  544629.566470  | 10.00, 0.00, 540170.00|\n| 0.00,-10.00, 3915...  37N   
4  510736.445518  | 10.00, 0.00, 510730.00|\n| 0.00,-10.00, 3948...  37N   

   zone_number hemisphere   epsg        lat       long  \
0           39          S  32739 -23.436720  49.009242   
1           37    

In [4]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, random_split

# Load CSV file
#csv_path = os.path.join(cwd, 'samples_2_.csv')
csv_path = '/home/ubuntu/work/satellite_data/sentinel_pauls_paper/samples.csv'
data = pd.read_csv(csv_path)

# Ensure necessary columns exist
assert 'path' in data.columns, "CSV must have a 'path' column"
assert 'latitudes' in data.columns and 'longitudes' in data.columns, "CSV must have 'latitudes' and 'longitudes' columns for geo splitting"

# Prepend the additional path to the file paths
data['path'] = '/home/ubuntu/work/satellite_data/sentinel_pauls_paper/samples/' + data['path']

In [5]:
# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]

In [6]:
# Randomly sample a subset of the original dataset
def create_subset(data, subset_size, seed=42):
    """
    Randomly selects a subset of the dataset.

    Parameters:
        data (DataFrame): Original dataset as a pandas DataFrame.
        subset_size (int): Number of elements to sample.
        seed (int): Seed for reproducibility.

    Returns:
        DataFrame: A randomly sampled subset of the dataset.
    """
    return data.sample(n=subset_size, random_state=seed).reset_index(drop=True)


In [7]:
# Define splitting functions
def split_data_torch(dataset, split_ratios=(0.8, 0.1, 0.1), sample_counts=None, generator=None):
    """
    Splits the data into train, val, and fix_val subsets using torch.utils.data.random_split.

    Parameters:
        dataset (Dataset): The PyTorch dataset.
        split_ratios (tuple): Ratios for train, val, and fix_val splits.
        sample_counts (tuple): Specific number of samples for train, val, and fix_val splits.
        generator (torch.Generator): A torch random generator for reproducibility.

    Returns:
        dict: A dictionary containing train, val, and fix_val subsets.
    """
    if sample_counts:
        lengths = sample_counts
    else:
        total_length = len(dataset)
        #total_length = sum(sample_counts)
        lengths = [int(r * total_length) for r in split_ratios]
        lengths[-1] = total_length - sum(lengths[:-1])  # Adjust for rounding errors

    train, val, fix_val = random_split(dataset, lengths, generator=generator)
    return {'train': train, 'val': val, 'fix_val': fix_val}

In [8]:
# Example usage
split_ratios = (0.8, 0.1, 0.1)  # For 80% train, 10% val, 10% fix_val
#sample_counts = (100, 20, 20)  # Specific number of samples

# Define the subset size
#subset_size = sum(sample_counts)  # Example: Use 140 elements from the original dataset
#data_subset = create_subset(data, subset_size)
data_subset = create_subset(data, len(data))

# Convert the DataFrame into a PyTorch Dataset
dataset = CustomDataset(data_subset)

In [9]:
# Random generator for reproducibility
generator = torch.Generator().manual_seed(42)

# Random split with ratios
splits_random = split_data_torch(dataset, split_ratios=split_ratios, generator=generator)

# Specific count split
#splits_count = split_data_torch(dataset, sample_counts=sample_counts, generator=generator)

In [ ]:
# Save splits to CSV files
data_subset.iloc[splits_random['train'].indices].to_csv('train.csv', index=False)
data_subset.iloc[splits_random['val'].indices].to_csv('val.csv', index=False)
data_subset.iloc[splits_random['fix_val'].indices].to_csv('fix_val.csv', index=False)

# Save subsets to CSV
#data_subset.iloc[splits_count['train'].indices].to_csv('train.csv', index=False)
#data_subset.iloc[splits_count['val'].indices].to_csv('val.csv', index=False)
#data_subset.iloc[splits_count['fix_val'].indices].to_csv('fix_val.csv', index=False)

print("Data saved to train.csv, val.csv, and fix_val.csv.")

'''
data.iloc[[idx for idx, _ in splits_count['train']]].to_csv('train.csv', index=False)
data.iloc[[idx for idx, _ in splits_count['val']]].to_csv('val.csv', index=False)
data.iloc[[idx for idx, _ in splits_count['fix_val']]].to_csv('fix_val.csv', index=False)

print("Data splitting completed and saved.")
'''

TypeError: Cannot index by location index with a non-integer key